In [1]:
import pandas as pd
import json
import typing as tp

from plotly import graph_objects as go

# pip install Pillow
from PIL.ImageColor import getcolor

In [8]:
def get_colors(fp, line_start, a_node, a_link, link_color_shift, color_step, cut):
    colors = []
    with open(fp, mode='r') as file:
        for line in file:
            colors.append('#' + line.strip())
    colors_node = []
    colors_link = []
    a_node = 1
    a_link = 0.5
    for i in range(line_start, len(colors)-cut, color_step):

        try:
            r, g, b = getcolor(colors[i+link_color_shift], "RGB")
            colors_link.append('rgba({r},{g},{b}, {a_link})'.format(r=r, g=g, b=b, a_link=a_link))

            r, g, b = getcolor(colors[i], "RGB")
            colors_node.append('rgba({r},{g},{b}, {a_node})'.format(r=r, g=g, b=b, a_node=a_node))  
        except IndexError:
            continue

    return colors_node, colors_link

In [7]:
list(range(1, 10, 1))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
duel_colors_node, duel_colors_link = get_colors(fp='/home/ivan/projects/visual_experiments/duel.txt',
                                      line_start=3,
                                      a_node=1,
                                      a_link=0.5,
                                      link_color_shift=1,
                                      color_step=8,
                                      cut=0)

In [10]:
dewdrop_colors_node, dewdrop_colors_link = get_colors(fp='/home/ivan/projects/visual_experiments/dewdrop-dynasty-40.txt',
                                      line_start=0,
                                      a_node=1,
                                      a_link=0.5,
                                      link_color_shift=1,
                                      color_step=2,
                                      cut=6)

In [3]:
raw_data = pd.read_excel('/home/ivan/projects/visual_experiments/data_sankey.xlsx')
raw_data = raw_data.to_json(indent=4, orient='records', force_ascii=False)
raw_data = json.loads(raw_data)
raw_data[:2]

[{'log_datetime': 1672531200000,
  'object_id': 'worker_1',
  'event_id': 1,
  'event_value': 100},
 {'log_datetime': 1672617600000,
  'object_id': 'worker_1',
  'event_id': 1,
  'event_value': 100}]

In [4]:
cond_data = pd.read_excel('/home/ivan/projects/visual_experiments/data_sankey_cond.xlsx', )
cond_data = cond_data.to_json(indent=4, orient='records', force_ascii=False)
cond_data = json.loads(cond_data)
cond_data

[{'event_id': 1,
  'event_value_condition': 100,
  'node_name': 'зачислен на обучение'},
 {'event_id': 2,
  'event_value_condition': 100,
  'node_name': 'закончил изучать теорию'},
 {'event_id': 3,
  'event_value_condition': 100,
  'node_name': 'закончил практику'},
 {'event_id': 4,
  'event_value_condition': 100,
  'node_name': 'сдал пробный экзамен'},
 {'event_id': 5,
  'event_value_condition': '>= 80',
  'node_name': 'сдал итоговый экзамен'},
 {'event_id': 5,
  'event_value_condition': '< 80',
  'node_name': 'завалил итоговый экзамен'}]

In [5]:
def prepare_data(raw_data, cond_data):
    '''
    raw_data = json[
        {
            log_datetime: datetime,
            object_id: str | int,
            event_id: str | int,
            event_value: int
        }, ...
    ]
    
    cond_data = json[
        {
            event_id: str | int,
            event_value_condition: str | int,
            node_name: str
        }, ...
    ]

    output = json[
        {
            object_id: str | int,
            step: int,
            source_node_name: str,
            target_node_name: str
        }, ...
    ]
    '''
    def _node_name_definition(raw_data_row):
        event_id_absence = True
        for cond in cond_data:
            if raw_data_row.iloc[2] == cond['event_id']:
                event_id_absence = False
                try:
                    comparison, value = cond['event_value_condition'].split(' ')
                    value = int(value)
                except (ValueError, AttributeError):
                    comparison = '='
                    value = int(cond['event_value_condition'])
                if comparison == '=' and raw_data_row.iloc[3] == value:
                    return cond['node_name']
                elif comparison == '>' and raw_data_row.iloc[3] > value:
                    return cond['node_name']
                elif comparison == '<' and raw_data_row.iloc[3] < value:
                    return cond['node_name']
                elif comparison == '>=' and raw_data_row.iloc[3] >= value:
                    return cond['node_name']
                elif comparison == '<=' and raw_data_row.iloc[3] <= value:
                    return cond['node_name']
                else:
                    continue
            else:
                continue
        if event_id_absence:
            return 'undefined due to event_id absence'
        else:
            return 'undefined due to event_condition_value absence'


    def _drop_duplicates(raw_data_df):
        raw_data_df = raw_data_df.sort_values(by=['object_id','log_datetime', 'node_name']).reset_index(drop=True)
        indexes_to_delete = []
        for i in range(1, len(raw_data_df)):
            if raw_data_df.iloc[i]['object_id'] == raw_data_df.iloc[i-1]['object_id']\
                and raw_data_df.iloc[i]['node_name'] == raw_data_df.iloc[i-1]['node_name']:
                indexes_to_delete.append(i)
        raw_data_df = raw_data_df.drop(index=indexes_to_delete)
        return raw_data_df


    def _source_target_definition(raw_data_df):
        prepared_data = raw_data_df.sort_values(by=['object_id','log_datetime', 'node_name']).reset_index(drop=True)
        prepared_data['step'] = prepared_data.groupby(['object_id']).cumcount() + 1
        prepared_data['source_node_name'] = prepared_data['node_name']
        prepared_data['target_node_name'] = prepared_data.groupby(['object_id'])['source_node_name'].shift(-1)
        prepared_data = prepared_data\
            .query('target_node_name.isna() == False')[['object_id', 'step', 'source_node_name', 'target_node_name']]
        return prepared_data
    
    raw_data_df = pd.json_normalize(raw_data)
    raw_data_df['node_name'] = raw_data_df.apply(_node_name_definition, axis=1)
    raw_data_df = _drop_duplicates(raw_data_df)
    prepared_data = _source_target_definition(raw_data_df)

    return prepared_data.to_json(indent=4, orient='records', force_ascii=False)
    

In [28]:
raw_data_json = json.loads(prepare_data(raw_data, cond_data))

raw_data_json= pd.json_normalize(raw_data_json)
raw_data_json = raw_data_json#.query('object_id == "worker_9"')
raw_data_json.head()

,object_id,step,source_node_name,target_node_name
0,worker_1,1,зачислен на обучение,закончил изучать теорию
1,worker_1,2,закончил изучать теорию,закончил практику
2,worker_1,3,закончил практику,сдал пробный экзамен
3,worker_1,4,сдал пробный экзамен,сдал итоговый экзамен
4,worker_10,1,зачислен на обучение,закончил изучать теорию


In [ ]:
def construct_sankey_skeleton(prepared_data):
    '''
    prepared_data: json = output from func prepare_data

    output: json = {
        "data":{\n
            "orientation": "h" | "v",\n
            "valueformat": ".0f" | str,\n
            "valuesuffix": str,\n
            "node": {
                "pad": int,
                "thickness": int,
                "line": {
                    "color": str,
                    "width": float
                    },
                "label": [str, ...],
                "color": ["rgba(0-255, 0-255, 0-255, 0.0 - 1.0)", ...]
                },\n
            "link": {
                "source": [int, ...],
                "target": [int, ...],
                "value": [float | int],
                "color": ["rgba(0-255, 0-255, 0-255, 0.0 - 1.0)", ...],
                "label": [optional[str], ...]
                }
            },\n
        "layout": {
            "title": {"text": str},
            "width": int,
            "height": int,
            "font": {"size": int}
            }
        }
    '''
    prepared_data_df = pd.json_normalize(prepared_data)

    def _prepare_lstv(prepared_data_df):
        label = []
        source = []
        target = []
        value = []

        # label
        object_id_prev = None
        for i in range(len(raw_data_json)):
            object_id = raw_data_json.iloc[i]['object_id']
            source_elem = raw_data_json.iloc[i]['source_node_name']
            target_elem = raw_data_json.iloc[i]['target_node_name']
            if object_id_prev != object_id:
                label_trimmed = label
                object_id_prev = object_id
            if source_elem not in(label_trimmed):
                label = [source_elem] + label
            if target_elem not in(label_trimmed):
                label = label + [target_elem]
                label_trimmed = label_trimmed + [target_elem]
            left_trim = label_trimmed.index(target_elem)
            label_trimmed = label_trimmed[left_trim:]

        # source, target, value
        object_id_prev = None
        result_dict = {}
        for i in range(len(raw_data_json)):
            object_id = raw_data_json.iloc[i]['object_id']
            source_elem = raw_data_json.iloc[i]['source_node_name']
            target_elem = raw_data_json.iloc[i]['target_node_name']
            if object_id_prev != object_id:
                position = 0
                object_id_prev = object_id
            source_indx = position + label[position:].index(source_elem)
            target_indx = source_indx + 1 + label[source_indx + 1:].index(target_elem)
            position = target_indx
            result_dict.setdefault((source_indx, target_indx), 0)
            result_dict[(source_indx, target_indx)] += 1

        for (source_elem, target_elem), value_elem in result_dict.items():
            source.append(source_elem)
            target.append(target_elem)
            value.append(value_elem)
        
        return label, source, target, value
